# INSERT statement

**SYNTAX**
```postgresql
INSERT INTO table1(column1, column2, …)
VALUES (value1, value2, …); 
```

The `INSERT` statement returns a command tag with the following form:
```postgresql
INSERT oid count
```
* The **OID** is an object identifier.
* PostgreSQL used the **OID** internally as a primary key for its system tables.
* Typically, the `INSERT` statement returns **OID** with a value of `0`.
* The count is the number of rows that the `INSERT` statement inserted successfully.

If you insert a new row into a table successfully, the return will typically look like:
```postgresql
INSERT 0 1
```

# RETURNING clause

The `INSERT` statement has an optional `RETURNING` clause that returns the information of the inserted row.

If you want to return the entire inserted row, you use an asterisk (`*`) after the `RETURNING` keyword:

```postgresql
INSERT INTO table1(column1, column2, …)
VALUES (value1, value2, …)
RETURNING *;
```

If you want to return some information about the inserted row, you can specify one or more columns after the `RETURNING` clause.

```postgresql
INSERT INTO table1(column1, column2, …)
VALUES (value1, value2, …)
RETURNING id;
```

To rename the returned value, you use the `AS` keyword followed by the name of the output.

```postgresql
INSERT INTO table1(column1, column2, …)
VALUES (value1, value2, …)
RETURNING output_expression AS output_name;
```



# INSERT Multiple Rows


```postgresql
INSERT INTO table_name (column_list)
VALUES
    (value_list_1),
    (value_list_2),
    ...
    (value_list_n);
```

To insert multiple rows and return the inserted rows, you add the `RETURNING` clause as follows:

```postgresql
INSERT INTO table_name (column_list)
VALUES
    (value_list_1),
    (value_list_2),
    ...
    (value_list_n)
RETURNING * | output_expression;
```

# UPDATE statement

```postgresql
UPDATE table_name
SET column1 = value1,
    column2 = value2,
    ...
WHERE condition;
```

When the `UPDATE` statement is executed successfully, it returns the following command tag:
```postgresql
UPDATE count
```
The count is the number of rows updated including rows whose values did not change.

# Returning updated rows

```postgresql
UPDATE table_name
SET column1 = value1,
    column2 = value2,
    ...
WHERE condition
RETURNING * | output_expression AS output_name;
```

# UPDATE JOIN

Sometimes, you need to update data in a table based on values in another table. 

In this case, you can use the PostgreSQL `UPDATE JOIN`.

**SYNTAX**:
```postgresql
UPDATE table1
SET table1.c1 = new_value
FROM table2
WHERE table1.c2 = table2.c2;
```

# DELETE statement

**SYNTAX**
```postgresql
DELETE FROM table_name
WHERE condition;
```
* The `WHERE` clause is optional.
* If you omit the `WHERE` clause, the `DELETE` statement will delete all rows in the table.
* The `DELETE` statement returns the number of rows deleted.
* It returns zero if the `DELETE` statement did not delete any row.

To return the deleted row(s) to the client, you use the `RETURNING` clause as follows:
```postgresql
DELETE FROM table_name
WHERE condition
RETURNING (select_list | *)
```

# DELETE JOIN (DELETE statement with USING clause)

* PostgreSQL does not support the `DELETE JOIN` statement. 
* Instead, it offers the `USING` clause in the `DELETE` statement that provides similar functionality to the `DELETE JOIN`.

```postgresql
DELETE FROM table1
USING table2
WHERE condition
RETURNING returning_columns;


DELETE FROM t1
USING t2
WHERE t1.id = t2.id
```

**EXAMPLE**:

The following statement uses the DELETE statement to delete all rows from the member table whose phones are in the denylist table:

```postgresql
DELETE FROM member
WHERE phone IN (
    SELECT
      phone
    FROM
      denylist
); 
```

# UPSERT Statement

* `UPSERT` is a combination of update and insert.
* The upsert allows you to update an existing row or insert a new one if it doesn’t exist.
* PostgreSQL does not have the `UPSERT` statement, but it supports the upsert operation via the **`INSERT...ON CONFLICT`** statement.
* If you use PostgreSQL 15 or later, you can use the **`MERGE`** statement, which is equivalent to the `UPSERT` statement.

**SYNTAX**:
```postgresql
INSERT INTO table_name (column1, column2, ...)
VALUES (value1, value2, ...)
ON CONFLICT (conflict_column)
DO NOTHING | DO UPDATE SET column1 = value1, column2 = value2, ...;
```
* `ON CONFLICT (conflict_column)`: This clause specifies the conflict target, which is the unique constraint or unique index that may cause a conflict.
* `DO NOTHING`: This instructs PostgreSQL to do nothing when a conflict occurs.
* `DO UPDATE`: This performs an update if a conflict occurs.

How the `INSERT ... ON CONFLICT` statement works.
* First, the `ON CONFLICT` clause identifies the conflict target, which is usually a unique constraint (or a unique index). If the data that you insert violates the constraint, a conflict occurs.
* Second, the `DO UPDATE` instructs PostgreSQL to update existing rows or do nothing rather than aborting the entire operation when a conflict occurs.
* Third, the `SET` clause defines the columns and values to update. You can use new values or reference the values you attempted to insert using the `EXCLUDED` keyword.

**EXAMPLE**:
```postgresql
INSERT INTO inventory (id, name, price, quantity)
VALUES (1, 'A', 16.99, 120)
ON CONFLICT(id)
DO UPDATE SET
  price = EXCLUDED.price,
  quantity = EXCLUDED.quantity;
```

**OUTPUT**: 
```postgresql
INSERT 0 1
```

* The `DO UPDATE` changes the price and quantity of the product to the new values being inserted. 
* The `EXCLUDED` allows you to access the new values.